In [1]:
import re
import numpy as np

class HashTable:

	# Create empty bucket list of given size
	def __init__(self, size):
		self.size = size
		self.hash_table = self.create_buckets()

	def create_buckets(self):
		return [[] for _ in range(self.size)]

	# Insert values into hash map
	def set_val(self, key, val):
		
		# Get the index from the key
		# using hash function
		hashed_key = hash(key) % self.size
		
		# Get the bucket corresponding to index
		bucket = self.hash_table[hashed_key]

		found_key = False
		for index, record in enumerate(bucket):
			record_key, record_val = record
			
			# check if the bucket has same key as
			# the key to be inserted
			if record_key == key:
				found_key = True
				break

		# If the bucket has same key as the key to be inserted,
		# Update the key value
		# Otherwise append the new key-value pair to the bucket
		if found_key:
			bucket[index] = (key, val)
		else:
			bucket.append((key, val))

	# Return searched value with specific key
	def get_val(self, key):
		
		# Get the index from the key using
		# hash function
		hashed_key = hash(key) % self.size
		
		# Get the bucket corresponding to index
		bucket = self.hash_table[hashed_key]

		found_key = False
		for index, record in enumerate(bucket):
			record_key, record_val = record
			
			# check if the bucket has same key as
			# the key being searched
			if record_key == key:
				found_key = True
				break

		# If the bucket has same key as the key being searched,
		# Return the value found
		# Otherwise indicate there was no record found
		if found_key:
			return record_val
		else:
			return "No record found"

	# Remove a value with specific key
	def delete_val(self, key):
		
		# Get the index from the key using
		# hash function
		hashed_key = hash(key) % self.size
		
		# Get the bucket corresponding to index
		bucket = self.hash_table[hashed_key]

		found_key = False
		for index, record in enumerate(bucket):
			record_key, record_val = record
			
			# check if the bucket has same key as
			# the key to be deleted
			if record_key == key:
				found_key = True
				break
		if found_key:
			bucket.pop(index)
		return

	# To print the items of hash map
	def __str__(self):
		return "".join(str(item) for item in self.hash_table)

In [2]:
fasta_file = "E:\\Studies\\4-1\\CSE 400\\drive\\outputs\\canu\\real_SAMN10819805_pacbio_00_canu_v1.9.fasta"
hash_table_contig=HashTable(100)
contigFile=open(fasta_file,"r")
line=contigFile.readline()
contig=""
contigsCount=0
contigName=""
contigLengthTotal=0
while line:
    if line[0]=='>':
        if contigsCount==0:
            contigName=re.split("\t| ",line)[0][1:].strip()
            contigsCount+=1
            line=contigFile.readline()

        else:
            hash_table_contig.set_val(contigName,contig)
            contigName=re.split("\t| ",line)[0][1:].strip()
            contigsCount+=1
            contigLengthTotal+=len(contig)
            contig=""
            line=contigFile.readline()
        continue
    line=line.strip()
    contig += line
    line=contigFile.readline()
contigFile.close()
hash_table_contig.set_val(contigName,contig)

In [3]:
mappedFile=open("mappedread.txt","r")
line=mappedFile.readline()
totalReadChecked=0
baseCountInRead=np.array([[0]*4]*4)
baseMatrix=[[0]*4]*4
baseMatrix=np.array(baseMatrix)
baseMatrix=baseMatrix.astype(int)
insertionCount=np.array([0]*100000)
deletionCount=np.array([0]*100000)
Mlen=0
Scount=0
Hcount=0
Icount=0
Ilen=0
Dcount=0
Dlen=0
totalReadLength=0
while line:
    totalReadChecked+=1
    line=line.strip()
    line=re.split("\t",line)
    flagg=line[1]
    contigName=line[2].strip()
    contigs1D=hash_table_contig.get_val(contigName)
    if contigs1D=="No record found":
        print(contigName)
        print("No record found")
        break
    strand=(int(flagg)&16)>>4
    position=int(line[3])-1
    ciger=line[4]
    cigerValues=re.split("S|M|I|D|H",ciger)
    cigerPosition=0

    rawRead=line[5]
    temp=rawRead.strip()
    baseCountInRead[0]+=rawRead.count('A')
    baseCountInRead[1]+=rawRead.count('T')
    baseCountInRead[2]+=rawRead.count('G')
    baseCountInRead[3]+=rawRead.count('C')


    totalReadLength+=len(rawRead)
    baseMatrixI=0
    baseMatrixJ=0

    readIterator=0
    contigPos=position
    differentCigerOperation=[]

    if(strand==1):
        rawRead=rawRead[::-1]
        rawRead=list(rawRead) 
        for i in range(len(rawRead)):
            if(rawRead[i]=='A'):
                rawRead[i]='T'
            elif rawRead[i]=='T':
                rawRead[i]='A'
            elif rawRead[i]=='G':
                rawRead[i]= 'C'
            elif rawRead[i]=='C':
                rawRead[i]= 'G'
        rawRead="".join(rawRead)
    for i in range(len(cigerValues)-1):
        value=int(cigerValues[i])
        cigerPosition+=len(cigerValues[i])
        cigerOperation=ciger[cigerPosition]
        cigerPosition+=1

        if cigerOperation=="M":
            Mlen+=value
            for it in range(value):
                fr=rawRead[readIterator]
                to=contigs1D[contigPos]
                if fr=="A":
                    baseMatrixI=0
                elif fr=="T":
                    baseMatrixI=1
                elif fr=="G":
                    baseMatrixI=2
                elif fr=="C":
                    baseMatrixI=3
                else:
                    baseMatrixI=4

                if to=="A":
                    baseMatrixJ=0
                elif to=="T":
                    baseMatrixJ=1
                elif to=="G":
                    baseMatrixJ=2
                elif to=="C":
                    baseMatrixJ=3
                else :
                    baseMatrixJ=4

                baseMatrix[baseMatrixI][baseMatrixJ]+=1

                readIterator+=1
                contigPos+=1
        elif cigerOperation=="I":
            readIterator+=value
            insertionCount[value]+=1
            Icount+=1
            Ilen+=value
        elif cigerOperation=="D":
            contigPos+=value
            deletionCount[value]+=1
            Dcount+=1
            Dlen+=value
        elif cigerOperation=="S":
            readIterator+=value
            Scount+=value
            deletionCount[value]+=1
            Dcount+=1
            Dlen+=value
        elif cigerOperation=="H":
            readIterator+=value
            Hcount+=value
            deletionCount[value]+=1
            Dcount+=1
            Dlen+=value
        else:
            differentCigerOperation.append(cigerOperation)

    line=mappedFile.readline()
mappedFile.close()

In [4]:
print(baseMatrix)

[[75901199   511207   771975   885033]
 [  507172 75894389   883785   770252]
 [  538624   840259 99606815  1038095]
 [  822495   539664  1039511 99724133]]


In [5]:
import pandas as pd
classes=["A","T","G","C"]
df=pd.DataFrame(baseMatrix,classes,classes)
print(df)
print("Total read check : ",totalReadChecked)
insertion2d=[]
deletion2d=[]
for i in range(10000):
    if insertionCount[i]==0:
        continue
    temp=[]
    temp.append(i)
    temp.append(insertionCount[i])
    insertion2d.append(temp)
for i in range(10000):
    if deletionCount[i]==0:
        continue
    temp=[]
    temp.append(i)
    temp.append(deletionCount[i])
    deletion2d.append(temp)
print("insertion  : ",len(insertion2d),"deletion : ",len(deletion2d))
print("Total Insertion Count : ",Icount)
print("Total Deletion Count : ",Dcount)
print("Total ReadLength: ",totalReadLength)




          A         T         G         C
A  75901199    511207    771975    885033
T    507172  75894389    883785    770252
G    538624    840259  99606815   1038095
C    822495    539664   1039511  99724133
Total read check :  60325
insertion  :  676 deletion :  8179
Total Insertion Count :  20681987
Total Deletion Count :  13434376
Total ReadLength:  531394770


In [6]:
df2=df.copy()
for i in range(4):
    df2.iloc[i]/=sum(df.iloc[i])
print(df2)



          A         T         G         C
A  0.972227  0.006548  0.009888  0.011336
T  0.006498  0.972312  0.011323  0.009868
G  0.005279  0.008236  0.976310  0.010175
C  0.008054  0.005284  0.010179  0.976483


In [7]:
insertionRate=Icount/Mlen
deletionRate=Dcount/Mlen
print(insertionRate)
print(deletionRate)

0.057406174459011555
0.03728926685835156


In [8]:

print("Total Read : ")
print("Total Mapped Read Count : ",totalReadChecked)
print("Total Read Length : ",totalReadLength)
print("Total Match/Missmatch length(M count) : ",Mlen)
print("Total Clipped Length(H/S count) : ",Scount+Hcount)
print("Total Insertion Length : ",Icount)
print("Total Deletion Length : ",Dcount)
print("Insertion Rate(Total InsLength/Total M count) : ",insertionRate)
print("Insertion Rate(Total InsLength/Total Read Length : ",Icount/totalReadLength)
print("Deletion Rate(Total delLength/M count : ",deletionRate)
print("Deletion Rate(Total delLength/Total Read Length : ",Dcount/totalReadLength)
print("Base Matrix : \n",df)
print("Probability(Row/sum of Row : \n",df2)





Total Read : 
Total Mapped Read Count :  60325
Total Read Length :  531394770
Total Match/Missmatch length(M count) :  360274608
Total Clipped Length(H/S count) :  136929760
Total Insertion Length :  20681987
Total Deletion Length :  13434376
Insertion Rate(Total InsLength/Total M count) :  0.057406174459011555
Insertion Rate(Total InsLength/Total Read Length :  0.03892019298571569
Deletion Rate(Total delLength/M count :  0.03728926685835156
Deletion Rate(Total delLength/Total Read Length :  0.02528134780099548
Base Matrix : 
           A         T         G         C
A  75901199    511207    771975    885033
T    507172  75894389    883785    770252
G    538624    840259  99606815   1038095
C    822495    539664   1039511  99724133
Probability(Row/sum of Row : 
           A         T         G         C
A  0.972227  0.006548  0.009888  0.011336
T  0.006498  0.972312  0.011323  0.009868
G  0.005279  0.008236  0.976310  0.010175
C  0.008054  0.005284  0.010179  0.976483


In [9]:
totalInsertionCount=sum(insertionCount)
totalInsertionLength=Ilen
totalDeletionLength=Dlen
totalDeletionCount=sum(deletionCount)
insertSizeMean=(float)(totalInsertionLength/ totalInsertionCount)
deletionSizeMean=(float)(totalDeletionLength/ totalDeletionCount)
inLengthDist=[]

for i in range(10000):
    inLengthDist.append(insertionCount[i]/totalInsertionCount)
delLengthDist=[]
for i in range(10000):
    delLengthDist.append(deletionCount[i]/totalDeletionCount)


In [10]:
print("Mean Insertion Size : ",insertSizeMean)
print("Mean deletion size : ",deletionSizeMean)

Mean Insertion Size :  1.65314880045133
Mean deletion size :  11.35775922901071


In [11]:
#convert dataframe to np array  

baseProbDF=np.array(df2)
print(baseProbDF)
#write base matrix to file
file=open("baseMatrix.txt","w")
for i in range(4):
    for j in range(4):
        file.write(str(baseProbDF[i][j])+"\t")
    file.write("\n")
file.close()

file=open("insertion.txt","w")
file.write(str(Ilen)+"\t"+str(Icount) +"\n")
for i in range(len(insertionCount)):
    file.write(str(insertionCount[i])+"\t")
file.write("\n")
file.close()

file=open("deletion.txt","w")
file.write(str(Dlen)+"\t"+str(Dcount)+"\n")
for i in range(len(deletionCount)):
    file.write(str(deletionCount[i])+"\t")
file.write("\n")
file.close()

file=open("parselog.txt","w")
file.write(str(Mlen))
file.close()


[[0.97222709 0.00654811 0.00988832 0.01133649]
 [0.00649757 0.97231193 0.01132251 0.00986799]
 [0.0052794  0.00823591 0.97630966 0.01017503]
 [0.00805374 0.00528431 0.01017873 0.97648322]]


# COMPUTE PROBABILITY